# How to finetune HuggingFace models on text data of any size and format with custom splitting (not random)

A way to handle text data of any size and format with custom split because random splitting is not recommended for protein sequences.

In [48]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
import torch
import numpy as np
from Bio import SeqIO
from datasets import Dataset, DatasetDict
from BioML.utilities import split_methods
from BioML.deep.embeddings import LLMConfig, TokenizeFasta
from BioML.utilities.utils import set_seed
from peft import get_peft_model, LoraConfig, TaskType
## https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb
## https://github.com/huggingface/notebooks/blob/main/examples/protein_language_modeling-tf.ipynb

ModuleNotFoundError: No module named 'datasets'

## Load data

You need to label the target values as labels so Trainer can recognize it.
Dataset can actually be used for any usecases with large   files it doesn't depend on transformers  
Although you would need to use PyTorch Dataloader to transform it into batches (but it only returns inputs ids and attention masks will it also return labels?)

In [49]:
def fasta_generator(fasta_file: str="../data/whole_sequence.fasta"):
    with open(fasta_file, 'r') as f:
        seqs = SeqIO.parse(f, 'fasta')
        for seq in seqs:
            yield {"id":seq.id, "seq":str(seq.seq)}

b = Dataset.from_generator(fasta_generator, gen_kwargs={"fasta_file":"../data/whole_sequence.fasta"})
y = np.random.randint(0, 2, size=len(b))
dataset = b.add_column("labels", y)

In [50]:
tok = TokenizeFasta()
tokens = tok.tokenize("../data/whole_sequence.fasta", (["labels", y],))

## Custom spliting with indices

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
cluster = split_methods.ClusterSpliter("../data/resultsDB_clu.tsv")
train, test = cluster.train_test_split(range(len(dataset)), index=dataset["id"])

In [53]:
train, test = train_test_split(range(len(dataset)), stratify=dataset["labels"], test_size=0.2) # random splitting

In [54]:
new = DatasetDict({"train":dataset.select(train), "test":dataset.select(test)})

In [55]:
train_, validation = cluster.train_test_split(range(len(new["train"])), index=new["train"]["id"])

In [56]:
new_2 = DatasetDict({"train":new["train"].select(train_), "test":dataset.select(test), "validation": new["train"].select(validation)})

## Load the protein language models model

In [57]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"

In [58]:
def model_init(): # 0 or 1 parameters ( the trial hyperparameters)
    return AutoModelForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", num_labels=2) # torch_dtype=torch.bfloat16 to load in bfloat16 which is accepted by CPUs unlike float16

def model_init2(): # 0 or 1 parameters ( the trial hyperparameters)
	return AutoModelForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", low_cpu_mem_usage=True)

model = AutoModelForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", num_labels=2, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
new["train"] = new["train"].map(lambda examples: tokenizer(examples["seq"], return_tensors="np",padding=True, truncation=True), batched=True)
new["test"] = new["test"].map(lambda examples: tokenizer(examples["seq"], return_tensors="np",padding=True, truncation=True), batched=True)

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

### Create the training arguments

In [60]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

In [61]:
lr = 8e-5
bs = 1
epochs = 4

Se use cpu to False whe you wan to use GPUs (it will automatically use GPUs), when f16 is True it will only use GPUs.

In [62]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.2, lr_scheduler_type='cosine', fp16=False if device=="cpu" else True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to=['mlflow'],
    load_best_model_at_end=True, metric_for_best_model="matthews_correlation", 
    save_total_limit=2, save_strategy="epoch", seed=3242342, gradient_accumulation_steps=4, use_cpu=True if device=="cpu" else False) 

## The warmup step together with cosine learning rate scheduler turns to onecycle learning rate scheduler
## weight decay for the Adam (AdamW) -> this is fast.Ai does
## fp16 is half precision -> mixed training (using fp32 and fp16)
## save_total_limit to 3 -> so only 3 models will be saved
## each 500 steps will be saved a model
## Save the report to mlflow
# How to evaluate mlflow?
# LR finder does not give reliable results for Transformers models https://github.com/huggingface/transformers/issues/16013

## Train the model using several evaluation metrics

In [45]:
import evaluate
import mlflow

You can use your own function as an evaluation metric -> then you have to retun as an dict  
Or you can use the evaluate library from hugging face to load different functions: [evaluate](https://huggingface.co/docs/evaluate/a_quick_tour)


In [46]:
def compute_classification_metrics(eval_pred):
    metrics = ["accuracy", "f1", "matthews_correlation", "precision", "recall"]
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    loaded = {metric:evaluate.load(metric) for metric in metrics}
    results = {metric: loaded[metric].compute(predictions=predictions, references=labels)[metric] 
               for metric in metrics}

    # the predictions from the models are logits (it also returns the labels, 
    # it also returns loss, attentions and hidden state but that is the classification model, for evalaution Trainer will only 
    # return logits and labels)
    return results

def compute_regression_metrics(eval_pred):
	metrics = ["mse", "mae"]
	logits, labels = eval_pred
	predictions = logits
	loaded = {metric:evaluate.load(metric) for metric in metrics}
	results = {metric: loaded[metric].compute(predictions=predictions, references=labels)[metric] 
			   for metric in metrics}
	results["r2"] = evaluate.load("r_squared").compute(predictions=predictions, references=labels)
	results["rmse"] = loaded["mse"].compute(predictions=predictions, references=labels, squared=False)["mse"]
	return results

In [63]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        print(inputs)
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.compute_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [19]:
trainer = Trainer(model, args, train_dataset=new['train'], eval_dataset=new['test'], # we need to pass tokenized datasets
                  tokenizer=tokenizer, compute_metrics=compute_classification_metrics, callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

c:\Users\ruite\miniforge3\envs\bioml\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Tuning the hyperparameters learning rate and batch_size

In [44]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [2, 4, 8, 16]),
    }


def compute_classification_objective(metrics: dict[str, float]) -> tuple[float, float]:
	return metrics["eval_loss"], metrics["eval_matthews_correlation"]

def compute_regression_objective(metrics: dict[str, float]) -> tuple[float, float]:
	return metrics["eval_loss"], metrics["eval_r2"]

In [67]:
trainer = Trainer(None, args, model_init=model_init2, train_dataset=new['train'], eval_dataset=new['test'], # we need to pass tokenized datasets
                  tokenizer=tokenizer, 
                  compute_metrics=compute_classification_metrics, 
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

RuntimeError: model_init should have 0 or 1 argument.

In [24]:
mlflow.end_run()

best_trials = trainer.hyperparameter_search(
	direction=["minimize", "maximize"],
	backend="optuna",
	hp_space=optuna_hp_space,
	n_trials=1,
	compute_objective=compute_classification_objective,
    storage='sqlite:///my_optuna_studies.db',
    load_if_exists=True
)


[I 2024-05-05 18:27:53,211] A new study created in RDB with name: no-name-305c8b7d-b552-4cd6-b1d9-2e327a424486
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/116 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

c:\Users\ruite\miniforge3\envs\bioml\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.6917587518692017, 'eval_accuracy': 0.5666666666666667, 'eval_f1': 0.0, 'eval_matthews_correlation': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 18.5898, 'eval_samples_per_second': 1.614, 'eval_steps_per_second': 0.807, 'epoch': 0.99}


...

{'eval_loss': 0.7241240739822388, 'eval_accuracy': 0.5, 'eval_f1': 0.4444444444444444, 'eval_matthews_correlation': -0.008988968316207744, 'eval_precision': 0.42857142857142855, 'eval_recall': 0.46153846153846156, 'eval_runtime': 17.8967, 'eval_samples_per_second': 1.676, 'eval_steps_per_second': 0.838, 'epoch': 1.98}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.7805582880973816, 'eval_accuracy': 0.5, 'eval_f1': 0.4444444444444444, 'eval_matthews_correlation': -0.008988968316207744, 'eval_precision': 0.42857142857142855, 'eval_recall': 0.46153846153846156, 'eval_runtime': 18.3333, 'eval_samples_per_second': 1.636, 'eval_steps_per_second': 0.818, 'epoch': 2.97}


[I 2024-05-05 18:36:15,996] Trial 0 finished with values: [0.7805582880973816, -0.008988968316207744] and parameters: {'learning_rate': 7.813286994811102e-05, 'gradient_accumulation_steps': 4}. 


{'train_runtime': 501.8248, 'train_samples_per_second': 0.933, 'train_steps_per_second': 0.231, 'train_loss': 0.6711522244859016, 'epoch': 2.97}


In [ ]:
trainer.args.learning_rate = best_trials[0].hyperparameters["learning_rate"]
trainer.args.gradient_accumulation_steps = best_trials[0].hyperparameters["gradient_accumulation_steps"]

In [ ]:
trainer.train()

  0%|          | 0/116 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.7221236824989319, 'eval_accuracy': 0.5333333333333333, 'eval_f1': 0.6956521739130436, 'eval_matthews_correlation': 0.0, 'eval_precision': 0.5333333333333333, 'eval_recall': 1.0, 'eval_runtime': 31.508, 'eval_samples_per_second': 0.952, 'eval_steps_per_second': 0.476, 'epoch': 0.99}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.7303746938705444, 'eval_accuracy': 0.5333333333333333, 'eval_f1': 0.6956521739130436, 'eval_matthews_correlation': 0.0, 'eval_precision': 0.5333333333333333, 'eval_recall': 1.0, 'eval_runtime': 31.2142, 'eval_samples_per_second': 0.961, 'eval_steps_per_second': 0.481, 'epoch': 1.98}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.7445932030677795, 'eval_accuracy': 0.5333333333333333, 'eval_f1': 0.6956521739130436, 'eval_matthews_correlation': 0.0, 'eval_precision': 0.5333333333333333, 'eval_recall': 1.0, 'eval_runtime': 31.2265, 'eval_samples_per_second': 0.961, 'eval_steps_per_second': 0.48, 'epoch': 2.97}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.753971517086029, 'eval_accuracy': 0.6, 'eval_f1': 0.7272727272727273, 'eval_matthews_correlation': 0.2857142857142857, 'eval_precision': 0.5714285714285714, 'eval_recall': 1.0, 'eval_runtime': 31.5365, 'eval_samples_per_second': 0.951, 'eval_steps_per_second': 0.476, 'epoch': 3.97}
{'train_runtime': 618.7594, 'train_samples_per_second': 0.756, 'train_steps_per_second': 0.187, 'train_loss': 0.6006371070598734, 'epoch': 3.97}


TrainOutput(global_step=116, training_loss=0.6006371070598734, metrics={'train_runtime': 618.7594, 'train_samples_per_second': 0.756, 'train_steps_per_second': 0.187, 'train_loss': 0.6006371070598734, 'epoch': 3.97})

We make sure that we get the same results by evaluating the results once more

In [ ]:
metrics = trainer.evaluate()
print(metrics)

## Search for hyperparameters like the learning rate which is the most important

Well it is actually batch size and learning rate -> smaller batch sizes tend to work better than large batch sizes -> but learning rate is affected by batch as well -> higher abtch need higher learning rate.

Fix everything else and tune the learning rate -> learning rate finder doesn'0t seem to work very well for transformers?  
But teh idea of learning rate finder is just test different learning rates -> so I cannot test them?

Ktrains: A wrapper to do many tasks and has a learning rate finder: [ktrains](https://github.com/amaiya/ktrain)

Use pytorch lightning perhaps: [pytorch_lighningt_huggingface](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/VisionTransformer/Fine_tuning_the_Vision_Transformer_on_CIFAR_10_with_PyTorch_Lightning.ipynb)

In [ ]:
model = AutoModel.from_pretrained("bigscience/T0pp", device_map="auto")

## Parameter efficient fine tuning

In [43]:
from functools import partial

In [ ]:
r_values = (16, 32, 64, 128, 256)

In [27]:
peft_config = LoraConfig(inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1, 
                         target_modules=["query", "value", "key", "output.dense"])

In [42]:
def get_target_module_names_for_peft(model, filter_="attention"):
    if isinstance(filter_, str):
        filter_ = [filter_] # if it is a string, convert it to a list
    module_names = []
    for name, module in model.named_modules():
        n = name.split(".")
        if filter_ and set(n).intersection(filter_):
            module_names.append(name)
        elif not filter_:
            module_names.append(name)
    return module_names

get_target_module_names_for_peft(model)

['base_model.model.esm.encoder.layer.0.attention',
 'base_model.model.esm.encoder.layer.0.attention.self',
 'base_model.model.esm.encoder.layer.0.attention.self.query',
 'base_model.model.esm.encoder.layer.0.attention.self.query.base_layer',
 'base_model.model.esm.encoder.layer.0.attention.self.query.lora_dropout',
 'base_model.model.esm.encoder.layer.0.attention.self.query.lora_dropout.default',
 'base_model.model.esm.encoder.layer.0.attention.self.query.lora_A',
 'base_model.model.esm.encoder.layer.0.attention.self.query.lora_A.default',
 'base_model.model.esm.encoder.layer.0.attention.self.query.lora_B',
 'base_model.model.esm.encoder.layer.0.attention.self.query.lora_B.default',
 'base_model.model.esm.encoder.layer.0.attention.self.query.lora_embedding_A',
 'base_model.model.esm.encoder.layer.0.attention.self.query.lora_embedding_B',
 'base_model.model.esm.encoder.layer.0.attention.self.key',
 'base_model.model.esm.encoder.layer.0.attention.self.key.base_layer',
 'base_model.model.

In [28]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 199,680 || all params: 8,040,443 || trainable%: 2.483445252954346


In [66]:
def get_lora_model(model, rank: int, target_modules: str | list[str], 
                    lora_alpha: int | None=None):
    
    if lora_alpha is None:
        lora_alpha = rank * 2
    else:
        print("Warning lora_alpha is set to a value. For optimal performance, it is recommended to set it double the rank")
    
    # get the lora models
    peft_config = LoraConfig(inference_mode=False, r=rank, lora_alpha=lora_alpha, lora_dropout=0.1, 
                                target_modules=target_modules)
    
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model

model_init2 = partial(get_lora_model, rank=8, target_modules=["query", "value", "key", "output.dense"])